In [1]:
# MATPLOTLIB INTERACTIVE VISUALIZATION. REMOVE (OR COMMENT) IF YOU NEED TO PRINT THE NOTEBOOK AS A PDF, SOMETIMES IT DOES NOT WORK WELL...
#%matplotlib widget

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm

from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import cross_val_score, ParameterGrid
from sklearn.metrics import make_scorer
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

# pd.options.display.max_rows = 9999
pd.options.display.max_columns = 200

#set initial random state
rs = 328537
np.random.seed(rs)

In [ ]:
DATASETPATH = "./data/DSL_Winter_Project_2024/DSL_Winter_Project_2024/development.csv"
dataset = pd.read_csv(DATASETPATH)

display(dataset)

In [ ]:
def generateColumnsNames(title, ignore=[]):
    out = []
    for i in range(0, 18):
        if(not i in ignore):
            out.append( title + "[" + str(i) + "]" )
    return out

In [ ]:
def generateColumnsNamesFromIndexes(indexes):
    out = []
    for index in indexes:
        for label in ["pmax", "negpmax", "area", "tmax", "rms"]:
            out.append(label + "[" + str(index) + "]")
    return out

regressionTargets = ['x', 'y']
noisyIndexes = [0, 7, 12, 15, 16, 17]
noisyFeatures= generateColumnsNamesFromIndexes(noisyIndexes)
featuresLabels = dataset.columns.drop(np.hstack([regressionTargets, noisyFeatures]))
dataset_shuff = dataset.sample(random_state=rs, frac=1)  #shuffle the dataset
X_df = dataset_shuff[featuresLabels]
Y_df = dataset_shuff[regressionTargets]

## Train - Test split

In [ ]:
ratio = 0.2
test_length = int(len(X_df) * 0.2)
train_length = len(X_df) - test_length
test_mask = np.array(test_length * [True] + train_length * [False])
np.random.shuffle(test_mask)
X_test = X_df.values[test_mask, :]
Y_test = Y_df.values[test_mask, :]
X_train = X_df.values[~test_mask, :]
Y_train = Y_df.values[~test_mask, :]

#tain-test split on a subset of features
X_red_df = X_df[generateColumnsNames("area", ignore=noisyIndexes) + generateColumnsNames("pmax", ignore=noisyIndexes) + generateColumnsNames("negpmax", ignore=noisyIndexes)]
X_test_red = X_red_df.values[test_mask, :]
X_train_red = X_red_df.values[~test_mask, :]

## Standardization

In [ ]:
scaler=StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

## Regression

In [ ]:
def euc_dist(Y, Y_pred):
    #return np.sqrt(((Y_pred - Y_test)**2).sum(axis=1)).sum(axis=0) / Y_pred.shape[0]
    return np.sqrt(((Y - Y_pred)**2).sum(axis=1)).mean()

euc_dist_scorer = make_scorer(euc_dist)

## Final result

In [ ]:
EVALUATIONPATH = "./data/DSL_Winter_Project_2024/DSL_Winter_Project_2024/evaluation.csv"
evaluation = pd.read_csv(EVALUATIONPATH, index_col="Id")
#X_ev = X_ev[generateColumnsNames("area", ignore=noisyIndexes) + generateColumnsNames("pmax", ignore=noisyIndexes)]
X_ev = evaluation[featuresLabels]

MLPReg.set_params(warm_start=False, verbose=True)
finalReg = MLPReg

stdScaler = StandardScaler()
X_std = stdScaler.fit_transform(X_df.values)
X_ev_std = stdScaler.transform(X_ev.values)

finalReg.fit(X_std, Y_df.values)
Y_ev = finalReg.predict(X_ev_std)


In [ ]:
MLPReg.get_params

In [ ]:
output = pd.DataFrame()
Y_ev_df = pd.DataFrame(Y_ev)
display(Y_ev_df)
#output['Predicted'] = (Y_ev_df[0]).round().astype(str) + "|" + (Y_ev_df[1]).round().astype(str)
output['Predicted'] = (Y_ev_df[0]).astype(str) + "|" + (Y_ev_df[1]).astype(str)
output.to_csv("submission10_MLP6.csv", index_label="Id")